In [1]:
import pandas as pd
import os
import requests
import pgeocode
from bs4 import BeautifulSoup


In [2]:
# IGJ vragenlijsten mogelijk te specificiek, met specialisten tot een consensus komen over een algemene score. Custom algoritme?
# Mogelijk 

In [3]:
# Download the data from the website
# The data is available at https://www.zorginzicht.nl/openbare-data
os.makedirs("data", exist_ok=True)
base_url = "https://www.zorginzicht.nl"
url = base_url + "/openbare-data"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
link_list = soup.find("ul", class_="link-list")
links = [a['href'].split("/")[2] for a in link_list.find_all("a", href=True)]
links = [link for link in links if "reval" in link]
links_c = [url + "/" + link for link in links]
# Remove links that contain "kwaliteitsstatuut"

for link in links_c:
    page = requests.get(link)
    soup = BeautifulSoup(page.content, "html.parser")
    # Find the div with class "page-module type-downloads"
    div = soup.find("div", class_="page-module type-downloads", id="verslagjaar-2023")
    link_file = base_url + div.find("a", class_ = "file-download icon-file-excel")['href']
    os.system(f'curl -o data/{"_".join(link.split("/")[-1].split("-")[2:])}.xlsx {link_file}')
    print(f'Downloaded {" ".join(link.split("/")[-1].split("-")[2:])}')

Downloaded medisch specialistische revalidatie msr


In [40]:
# Read the data from the downloaded files into a single DataFrame
# retrieve the second sheet from the file
dfs = {}
# Loop through all the files in the data directory and read the second sheet of each file into the df
for file in os.listdir("data"):
    if file.endswith(".xlsx"):
        # Read the sheet names from the file
        xls = pd.ExcelFile(os.path.join("data", file))
        sheet_names = xls.sheet_names
        print(sheet_names)
        # find the index of the sheet name that contains "2023"
        sheet_name = [sheet for sheet in sheet_names if "2023" in sheet]
        print(sheet_name)
        dfs[('_').join(file.split('.')[0].split('_')[2:])] = pd.read_excel(xls, sheet_name=sheet_name[0], header=0)
        
        # If the sheet name is not found, try reading the first sheet
        # Append the data to the df DataFrame
        #df = pd.concat([df, df_temp], ignore_index=True)
if len(dfs) == 1:
    dfs = dfs[list(dfs.keys())[0]]


['MSR Verslagjaar 2023', 'Begrippenlijst']
['MSR Verslagjaar 2023']


In [44]:
kvks = dfs['KvkNummer'].unique()
# drop dfs rows where IndicatorNaam doesn't contain 'rapport'
dfs = dfs[dfs['IndicatorNaam'].str.contains('rapport')].reset_index(drop=True)
len(dfs)

147

In [47]:
dfs.head(5)

,Sector,Verslagjaar,OrganisatieNaam,KvkNummer,OrganisatieAGBCode,LocatieNaam,Vestigingsnummer,LocatiePostcode,LocatiePlaats,LocatieHuisnummer,...,IndicatorWaarde,Teller,Noemer,Aantal waarnemingen (N),Indicator_nvt,Opmerking,Bron,MeetperiodeBeginDatum,MeetperiodeEindDatum,AanleverFrequentie
0,Medisch-specialistische revalidatie,2023,Stichting Revalidatie Friesland,41000436,6160201,"Revalidatie Friesland, concern",27820491,9244CL,Beetsterzwaag,3,...,8.6,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
1,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.7,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
2,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.9,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
3,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.4,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
4,Medisch-specialistische revalidatie,2023,Reade,34321379,6160706,"Reade, concern",20869916,1054HW,Amsterdam,283,...,8.8,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks


In [35]:
dfs = dfs[['OrganisatieNaam', 'KvkNummer', 'LocatieNaam', 'LocatiePostcode', 'IndicatorNaam', 'IndicatorWaarde', 'Indicator_nvt']]
# find all OrganisatieNaam where there are multiple LocatieNaam
dfs

,OrganisatieNaam,KvkNummer,LocatieNaam,LocatiePostcode,IndicatorNaam,IndicatorWaarde,Indicator_nvt
0,Stichting Revalidatie Friesland,41000436,"Revalidatie Friesland, concern",9244CL,Volwassenen: Wat is het gemiddelde rapportcijf...,8.6,0
1,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,"Stichting Revalidatiecentrum ""Het Roessingh""",7522AH,Volwassenen: Wat is het gemiddelde rapportcijf...,8.7,0
2,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,"Stichting Revalidatiecentrum ""Het Roessingh""",7522AH,Kinderen: Wat is het gemiddelde rapportcijfer ...,8.9,0
3,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,"Stichting Revalidatiecentrum ""Het Roessingh""",7522AH,Jeugd: Wat is het gemiddelde rapportcijfer dat...,8.4,0
4,Reade,34321379,"Reade, concern",1054HW,Volwassenen: Wat is het gemiddelde rapportcijf...,8.8,0
...,...,...,...,...,...,...,...
142,Hand- en Pols Revalidatie Nederland B.V.,56456549,Stichting Hand- en Pols Revalidatie Nederland,4461GL,Kinderen: Wat is het gemiddelde rapportcijfer ...,NaN,1
143,Hand- en Pols Revalidatie Nederland B.V.,56456549,Stichting Hand- en Pols Revalidatie Nederland,4461GL,Jeugd: Wat is het gemiddelde rapportcijfer dat...,NaN,1
144,BeLife,58514856,Concernniveau: BeLife,3062MA,Volwassenen: Wat is het gemiddelde rapportcijf...,8.70,0
145,BeLife,58514856,Concernniveau: BeLife,3062MA,Kinderen: Wat is het gemiddelde rapportcijfer ...,NaN,1


In [64]:
dfs

,Sector,Verslagjaar,OrganisatieNaam,KvkNummer,OrganisatieAGBCode,LocatieNaam,Vestigingsnummer,LocatiePostcode,LocatiePlaats,LocatieHuisnummer,...,IndicatorWaarde,Teller,Noemer,Aantal waarnemingen (N),Indicator_nvt,Opmerking,Bron,MeetperiodeBeginDatum,MeetperiodeEindDatum,AanleverFrequentie
0,Medisch-specialistische revalidatie,2023,Stichting Revalidatie Friesland,41000436,6160201,"Revalidatie Friesland, concern",27820491,9244CL,Beetsterzwaag,3,...,8.6,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
1,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.7,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
2,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.9,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
3,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.4,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
4,Medisch-specialistische revalidatie,2023,Reade,34321379,6160706,"Reade, concern",20869916,1054HW,Amsterdam,283,...,8.8,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Medisch-specialistische revalidatie,2023,Hand- en Pols Revalidatie Nederland B.V.,56456549,22220459,Stichting Hand- en Pols Revalidatie Nederland,1,4461GL,Goes,35,...,NaN,NaN,NaN,NaN,1,Niet op te splitsen naar leeftijd,NaN,2023-01-01,2023-12-31,Jaarlijks
143,Medisch-specialistische revalidatie,2023,Hand- en Pols Revalidatie Nederland B.V.,56456549,22220459,Stichting Hand- en Pols Revalidatie Nederland,1,4461GL,Goes,35,...,NaN,NaN,NaN,NaN,1,Niet op te splitsen naar leeftijd,NaN,2023-01-01,2023-12-31,Jaarlijks
144,Medisch-specialistische revalidatie,2023,BeLife,58514856,22220664,Concernniveau: BeLife,27850951,3062MA,ROTTERDAM,72,...,8.70,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks
145,Medisch-specialistische revalidatie,2023,BeLife,58514856,22220664,Concernniveau: BeLife,27850951,3062MA,ROTTERDAM,72,...,NaN,NaN,NaN,NaN,1,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks


In [ ]:
ap = dfs.groupby(['IndicatorNaam', 'Indicator_nvt']).size().sort_values(ascending=False)
# take all indicatornaam that contain rapport
ap = ap[ap.index.get_level_values(0).str.contains('rapport')]

IndicatorNaam                                                          Indicator_nvt
Volwassenen: Wat is het gemiddelde rapportcijfer dat patiënten geven?  0                50
Jeugd: Wat is het gemiddelde rapportcijfer dat ouders geven?           1                37
Kinderen: Wat is het gemiddelde rapportcijfer dat ouders geven?        1                33
                                                                       0                14
Jeugd: Wat is het gemiddelde rapportcijfer dat ouders geven?           0                11
Volwassenen: Wat is het gemiddelde rapportcijfer dat patiënten geven?  1                 2
dtype: int64

In [ ]:
dfs

In [104]:
df_subset = dfs[dfs['IndicatorNaam'].str.contains('Volwassen')]
df_subset = df_subset[df_subset['Indicator_nvt'] == 0]
df_subset.columns

Index(['Sector', 'Verslagjaar', 'OrganisatieNaam', 'KvkNummer',
       'OrganisatieAGBCode', 'LocatieNaam', 'Vestigingsnummer',
       'LocatiePostcode', 'LocatiePlaats', 'LocatieHuisnummer',
       'LocatieAGBCode', 'Type Zorgaanbieder', 'IndicatorsetCode',
       'IndicatorsetNaam', 'Thema', 'IndicatorCode', 'IndicatorNummer',
       'IndicatorNaam', 'IndicatorEenheid', 'IndicatorType', 'IndicatorWaarde',
       'Teller', 'Noemer', 'Aantal waarnemingen (N)', 'Indicator_nvt',
       'Opmerking', 'Bron', 'MeetperiodeBeginDatum', 'MeetperiodeEindDatum',
       'AanleverFrequentie'],
      dtype='object')

In [132]:
dfs

,Sector,Verslagjaar,OrganisatieNaam,KvkNummer,OrganisatieAGBCode,LocatieNaam,Vestigingsnummer,LocatiePostcode,LocatiePlaats,LocatieHuisnummer,...,IndicatorWaarde,Teller,Noemer,Aantal waarnemingen (N),Indicator_nvt,Opmerking,Bron,MeetperiodeBeginDatum,MeetperiodeEindDatum,AanleverFrequentie
0,Medisch-specialistische revalidatie,2023,Stichting Revalidatie Friesland,41000436,6160201,"Revalidatie Friesland, concern",27820491,9244CL,Beetsterzwaag,3,...,8.6,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
1,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.7,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
2,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.9,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
3,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,8.4,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
4,Medisch-specialistische revalidatie,2023,Reade,34321379,6160706,"Reade, concern",20869916,1054HW,Amsterdam,283,...,8.8,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Medisch-specialistische revalidatie,2023,Hand- en Pols Revalidatie Nederland B.V.,56456549,22220459,Stichting Hand- en Pols Revalidatie Nederland,1,4461GL,Goes,35,...,NaN,NaN,NaN,NaN,1,Niet op te splitsen naar leeftijd,NaN,2023-01-01,2023-12-31,Jaarlijks
143,Medisch-specialistische revalidatie,2023,Hand- en Pols Revalidatie Nederland B.V.,56456549,22220459,Stichting Hand- en Pols Revalidatie Nederland,1,4461GL,Goes,35,...,NaN,NaN,NaN,NaN,1,Niet op te splitsen naar leeftijd,NaN,2023-01-01,2023-12-31,Jaarlijks
144,Medisch-specialistische revalidatie,2023,BeLife,58514856,22220664,Concernniveau: BeLife,27850951,3062MA,ROTTERDAM,72,...,8.70,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks
145,Medisch-specialistische revalidatie,2023,BeLife,58514856,22220664,Concernniveau: BeLife,27850951,3062MA,ROTTERDAM,72,...,NaN,NaN,NaN,NaN,1,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks


In [121]:
dfs[[df_subset.index]] = df_subset

ValueError: Columns must be same length as key

In [133]:
import xml.etree.ElementTree as ET

df = pd.DataFrame()
for Leeftijd in ['Jeugd','Volwassenen', 'Kinderen']:
    df_subset = dfs[dfs['IndicatorNaam'].str.contains('Volwassen')]
    df_subset = df_subset[df_subset['Indicator_nvt'] == 0]
    loc = []
    url = "https://zorgbeeld.nza.nl/openapi/WaitingTimeMSZ"
    for kvk,pc in zip(df_subset['KvkNummer'], df_subset['LocatiePostcode']):
        print(f'KVK: {kvk} - pc: {pc}')
        params = {
            "KVKNummer": kvk
        }

        response = requests.get(url, params=params)
        i = 0
        if response.status_code == 200:
            root = ET.fromstring(response.content)
            for entry in root.findall("TL_REST"):
                postal_code = entry.findtext("PostalCode")
                waiting_time = entry.findtext("WaitingTime")
                if postal_code == pc and waiting_time is not None:
                    loc.append(waiting_time)
                    print(f'{postal_code} - {waiting_time}')
                    i = 1
                    break
            if i == 0:
                print('No Match found')
                loc.append(None)
        else:
            print(f"Error: {response.status_code} for KVK {kvk}")
            loc.append(None)
    df_subset['loc'] = loc
    df = pd.concat([df, df_subset])

KVK: 41000436 - pc: 9244CL
9244CL - 28
KVK: 41027866 - pc: 7522AH
7522AH - 4
KVK: 34321379 - pc: 1054HW
1054HW - 26
KVK: 41127056 - pc: 3015LJ
3015LJ - 58
KVK: 20136126 - pc: 4817JW
4817JW - 14
KVK: 41049838 - pc: 6813GG
6813GG - 27
KVK: 41149113 - pc: 3583TM
3583TM - 39
KVK: 28099814 - pc: 2543SW
2543SW - 25
KVK: 41022329 - pc: 8013XZ
8013XZ - 29
KVK: 18065595 - pc: 5629CC
5629CC - 7
KVK: 41194854 - pc: 1213VX
1213VX - 9
KVK: 41125312 - pc: 3062MA
3062MA - 8
KVK: 14127976 - pc: 6411GW
6411GW - 30
KVK: 41055111 - pc: 6574NA
6574NA - 73
KVK: 1169570 - pc: 9751ND
No Match found
KVK: 41072650 - pc: 6432CC
6432CC - 16
KVK: 62375644 - pc: 6419PC
6419PC - 48
KVK: 27370985 - pc: 3941PW
3941PW - 1
KVK: 1170406 - pc: 7909AA
No Match found
KVK: 24299846 - pc: 3079DZ
3079DZ - 26
KVK: 30244197 - pc: 3584CX
3584CX - 56
KVK: 32082917 - pc: 3813TZ
3813TZ - 104
KVK: 41035024 - pc: 3844DG
3844DG - 2
KVK: 41066359 - pc: 6043CV
6043CV - 5
KVK: 41084046 - pc: 5223GZ
5223GZ - 90
KVK: 5075996 - pc: 7772SE
N

In [136]:
df.dropna(subset=['loc'], inplace=True)

# save as csv
df.to_csv('data/WaitingTimes.csv', index=False)


In [5]:
df

,Sector,Verslagjaar,OrganisatieNaam,KvkNummer,OrganisatieAGBCode,LocatieNaam,Vestigingsnummer,LocatiePostcode,LocatiePlaats,LocatieHuisnummer,...,Teller,Noemer,Aantal waarnemingen (N),Indicator_nvt,Opmerking,Bron,MeetperiodeBeginDatum,MeetperiodeEindDatum,AanleverFrequentie,loc
0,Medisch-specialistische revalidatie,2023,Stichting Revalidatie Friesland,41000436,6160201,"Revalidatie Friesland, concern",27820491,9244CL,Beetsterzwaag,3,...,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks,28
1,Medisch-specialistische revalidatie,2023,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,6160401,"Stichting Revalidatiecentrum ""Het Roessingh""",41027866,7522AH,Enschede,33,...,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks,4
2,Medisch-specialistische revalidatie,2023,Reade,34321379,6160706,"Reade, concern",20869916,1054HW,Amsterdam,283,...,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks,26
3,Medisch-specialistische revalidatie,2023,Rijndam Revalidatie,41127056,6160808,Rijndam Revalidatie,28240588,3015LJ,Rotterdam,300,...,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks,58
4,Medisch-specialistische revalidatie,2023,Stichting Revant,20136126,6161007,"Revant, concern",23394110,4817JW,Breda,1,...,NaN,NaN,NaN,0,NaN,Expoints,2023-01-01,2023-12-31,Jaarlijks,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Medisch-specialistische revalidatie,2023,Stichting Medinello Revalidatie Zorg,53019342,22220441,Concernniveau: Stichting Medinello Revalidatie...,43464165,3821AL,Amersfoort,2 A,...,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks,7
94,Medisch-specialistische revalidatie,2023,Stichting Revalidatiegeneeskunde Nederland,59943106,22220690,Stichting Revalidatiegeneeskunde Nederland,30679540,4461PK,GOES,53,...,NaN,NaN,NaN,0,Wij behandelen geen kinderen en/of jeugd. Wij ...,NaN,2023-01-01,2023-12-31,Jaarlijks,12
95,Medisch-specialistische revalidatie,2023,Stichting Revalide,60806702,22220699,Revalide Helmond B.V.,29956366,5701AH,Helmond,131,...,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks,26
96,Medisch-specialistische revalidatie,2023,Stichting Spine and Joint Centre the Netherlands,41135552,6160809,Concernniveau: Stichting Spine and Joint Centr...,1,3035EM,Rotterdam,113,...,NaN,NaN,NaN,0,NaN,NaN,2023-01-01,2023-12-31,Jaarlijks,107


In [6]:
# Keep only the relevant columns
import pandas as pd
df = pd.read_csv('WaitingTimes.csv', sep=',', encoding='utf-8')
df = df[["OrganisatieNaam", "LocatiePlaats", "LocatiePostcode", "IndicatorWaarde", "Indicator_nvt", "IndicatorNaam", 'loc']]

In [13]:
# take only the first 4 number of postal code
df['LocatiePostcode'] = df['LocatiePostcode'].apply(lambda x: x[:4])

In [74]:
import pgeocode
postcode = 1183
nomi = pgeocode.GeoDistance('NL')

score1 = -(nomi.query_postal_code(postcode, df['LocatiePostcode'].values))/25
score2 = df['IndicatorWaarde']
score3 = -df['loc']/5
score = score1 + score2 + score3
# locate the highest score index
index = score.idxmax()
print(f'Best match')
print(f'Waiting time: {df.iloc[index]["loc"]} Day(s)')
print(f'OrganistatieNaam: {df.iloc[index]["OrganisatieNaam"]}')
print(f'Postcode: {df.iloc[index]["LocatiePostcode"]}')
print(f'Plaats: {df.iloc[index]["LocatiePlaats"]}')
print(f'Kwaliteit: {df.iloc[index]["IndicatorWaarde"]}')
print(f'Distance: {nomi.query_postal_code(postcode, df.iloc[index]["LocatiePostcode"])} KM')
# allemaal in één lijn

Best match
Waiting time: 1 Day(s)
OrganistatieNaam: Ministerie van Defensie
Postcode: 3941
Plaats: Doorn
Kwaliteit: 8.9
Distance: 43.70776164201557 KM
